In [19]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

## Data partition

In [20]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [21]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    555     555 

## Models

In [22]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [23]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size=10, mtry=42, splitrule=gini 
- Fold01.Rep1: min.node.size=10, mtry=42, splitrule=gini 
+ Fold01.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
+ Fold01.Rep1: min.node.size= 3, mtry=42, splitrule=gini 
- Fold01.Rep1: min.node.size= 3, mtry=42, splitrule=gini 
+ Fold02.Rep1: min.node.size=10, mtry=42, splitrule=gini 
- Fold02.Rep1: min.node.size=10, mtry=42, splitrule=gini 
+ Fold02.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
+ Fold02.Rep1: min.node.size= 3, mtry=42, splitrule=gini 
- Fold02.Rep1: min.node.size= 3, mtry=42, splitrule=gini 
+ Fold03.Rep1: min.node.size=10, mtry=42, splitrule=gini 
- Fold03.Rep1: min.node.size=10, mtry=42, splitrule=gini 
+ Fold03.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
- Fold03.Rep1: min.node.size= 5, mtry=28, splitrule=extratrees 
+ Fold03.Rep1: min.node.size= 3, mtr

In [24]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

+ Fold01.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
- Fold01.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
+ Fold01.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
- Fold01.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
+ Fold01.Rep1: mtry=6, coefReg=0.1526, coefImp=0.89319 
- Fold01.Rep1: mtry=6, coefReg=0.1526, coefImp=0.89319 
+ Fold02.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
- Fold02.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
+ Fold02.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
- Fold02.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
+ Fold02.Rep1: mtry=6, coefReg=0.1526, coefImp=0.89319 
- Fold02.Rep1: mtry=6, coefReg=0.1526, coefImp=0.89319 
+ Fold03.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
- Fold03.Rep1: mtry=2, coefReg=0.1695, coefImp=0.04701 
+ Fold03.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
- Fold03.Rep1: mtry=4, coefReg=0.7036, coefImp=0.20474 
+ Fold03.Rep1: mtry=6, coefReg=0.1526, coefImp=0.89319 
- Fold03.Rep1: mtry=6, coefReg=0.1526, coefImp=0

## Results

In [25]:
print(model_rf)
plot(model_rf)

Random Forest 

1110 samples
  73 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 999, 999, 999, 999, 999, 999, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
   3             42    gini        0.7837800  0.5674352
   5             28    extratrees  0.7864505  0.5727782
  10             42    gini        0.7765320  0.5529188

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 28, splitrule = extratrees
 and min.node.size = 5.


In [26]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

1110 samples
  73 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 998, 999, 998, 999, 999, 1000, ... 
Resampling results across tuning parameters:

  mtry  coefReg    coefImp     Accuracy   Kappa    
  2     0.1694588  0.04701206  0.6992034  0.3983268
  4     0.7036451  0.20473636  0.7613433  0.5227360
  6     0.1525607  0.89319130  0.7116697  0.4233980

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 4, coefReg = 0.7036451
 and coefImp = 0.2047364.


In [27]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.  1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.7477477 0.766421 0.7837838 0.7864505 0.8090010 0.8288288    0
rrf    0.7053571 0.740991 0.7522523 0.7613433 0.7900491 0.8181818    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.4949626 0.5328145 0.5673276 0.5727782 0.6177483 0.6579075    0
rrf    0.4107143 0.4820035 0.5045095 0.5227360 0.5804259 0.6363636    0


## Prediction

In [28]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     117      54
   NotFast   53     252
                                        
               Accuracy : 0.7752        
                 95% CI : (0.735, 0.812)
    No Information Rate : 0.6429        
    P-Value [Acc > NIR] : 2.931e-10     
                                        
                  Kappa : 0.5111        
                                        
 Mcnemar's Test P-Value : 1             
                                        
            Sensitivity : 0.6882        
            Specificity : 0.8235        
         Pos Pred Value : 0.6842        
         Neg Pred Value : 0.8262        
             Prevalence : 0.3571        
         Detection Rate : 0.2458        
   Detection Prevalence : 0.3592        
      Balanced Accuracy : 0.7559        
                                        
       'Positive' Class : fast          
                                        

Sensitivity          Specificity       Pos Pred Value 
           0.6882353            0.8235294            0.6842105 
      Neg Pred Value            Precision               Recall 
           0.8262295            0.6842105            0.6882353 
                  F1           Prevalence       Detection Rate 
           0.6862170            0.3571429            0.2457983 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.7558824

In [29]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [30]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 171 controls (TestClasses fast) < 305 cases (TestClasses NotFast).
Area under the curve: 0.7552

In [31]:
pred_rf.prob

fast,NotFast
0.790633333,0.2093667
0.167633333,0.8323667
0.869966667,0.1300333
0.357566667,0.6424333
0.272400000,0.7276000
0.969400000,0.0306000
0.824633333,0.1753667
0.791433333,0.2085667
0.032133333,0.9678667
0.326633333,0.6733667
